In [41]:
import json
import logging
import pandas as pd
from time import sleep
from SPARQLWrapper import SPARQLWrapper, JSON

In [42]:
#languages = ["de", "en"]
languages = ["en"]

In [43]:
def execute(query: str, endpoint_url: str = 'https://dbpedia.org/sparql'):
    """
    https://dbpedia.org/sparql
    https://query.wikidata.org/bigdata/namespace/wdq/sparql
    """
    agent_header = {'User-Agent': 'wiki_parser_online/0.17.1 (https://deeppavlov.ai;'
                                        ' info@deeppavlov.ai) deeppavlov/0.17.1'}
    timeout = 1
    e = ''
    while timeout < 3:
        try:
            sparql = SPARQLWrapper(endpoint_url)
            # sparql.agent = str(agent_header)
            sparql.setQuery(query)
            sparql.setReturnFormat(JSON)
            response = sparql.query().convert()
            return response
        except Exception as e:
            logging.error(str(e))
            sleep(timeout)
            e = str(e)
            if 'MalformedQueryException' in e or 'bad formed' in e:
                logging.error(query)
                return {'error': 'bad formed query'}            
            timeout += 1
    
    return {'error': 'out of the loop'}

def save_json(data, path):
    with open(path, 'w') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

def load_json(path):
    with open(path, 'r') as f:
        return json.load(f)

label_query = """
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
SELECT ?label
WHERE {{
    <{uri}> rdfs:label ?label .
    FILTER (lang(?label) = '{lang}')    

    <{uri}> rdfs:label ?label_en .
    FILTER (lang(?label_en) = 'en')    
    <{uri}> rdfs:label ?label_de .
    FILTER (lang(?label_de) = 'de')    
    <{uri}> rdfs:label ?label_ru .
    FILTER (lang(?label_ru) = 'ru')    
}}
"""

sameAs_query = """
    PREFIX owl: <http://www.w3.org/2002/07/owl#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX dbo: <http://dbpedia.org/ontology/>
    SELECT *
    WHERE {{
        <{uri}> dbo:wikiPageRedirects*/owl:sameAs ?sameAs .
        FILTER(contains(str(?sameAs), "wikidata")  )
    }}
"""

# Translate to another language


In [44]:
request_cache = {}

In [45]:
translations = {}
translations["de"] = {
    "Museum":                                       "Museums", 
    "Natural disaster":                             "Naturkatastrophen",
    "Beverage":                                     "Getränke",
    "Active political party":                       "Aktive politische Parteien",
    "ColdWar politician":                           "Politiker/-innen im Kalten Krieg",
    "WWII politician":                              "Politiker im 2. Weltkrieg",
    "Active political parties(last 20 years)":      "Aktive politische Parteien (der letzten 20 Jahre)",
    "Film festival":                                "Filmfestivals",
    "Soccer club":                                  "Fußballvereine",
    "Newspaper":                                    "Zeitungen",
    "Male actor born after 1950":                   "Schauspieler, geboren nach 1950",
    "Magazine":                                     "Zeitschriften",
    "Company":                                      "Unternehmen",
    "Lake with the area more than 100 square km":   "Seen mit einer Fläche von mehr als 100 km²",
    "Mountain higher than 4000m":                   "Berge höher als 4000 Meter",
    "Nominee of the Academy Award for Best Actor":  "Nominierte für den Oscar für den besten Schauspieler",
    "News presenter":                               "Nachrichtenmoderatoren/-innen"
}
translations["en"] = {
    "Museum":                                       "Museums", 
    "Natural disaster":                             "Natural disasters",
    "Beverage":                                     "Beverages",
    "Active political party":                       "Active political parties",
    "ColdWar politician":                           "Politicians during the Cold War",
    "WWII politician":                              "Politicians during World War II",
    "Active political parties(last 20 years)":      "Active political parties (last 20 years)",
    "Film festival":                                "Film festivals",
    "Soccer club":                                  "Soccer clubs",
    "Newspaper":                                    "Newspapers",
    "Male actor born after 1950":                   "Male actors born after 1950",
    "Magazine":                                     "Magazines",
    "Company":                                      "Companies",
    "Lake with the area more than 100 square km":   "Lakes with the area more than 100 square km",
    "Mountain higher than 4000m":                   "Mountains higher than 4000 meters",
    "Nominee of the Academy Award for Best Actor":  "Nominees of the Academy Award for Best Actor",
    "News presenter":                               "News presenter"
}

def get_translation_of_topic(language, topic):
    return translations.get(language).get(topic)

In [46]:


def get_label_from_wikidata(wikidata_uri, language):
    sparql = label_query.format(uri=wikidata_uri, lang=language)
    result = execute(sparql, endpoint_url="https://query.wikidata.org/bigdata/namespace/wdq/sparql")
    try:
        return result['results']['bindings'][0]['label']['value']
    except Exception as e:
        print("no", language, "label for", wikidata_uri, "\n", sparql)
        raise e


In [47]:
df = pd.read_csv('../data/toloka/test-topics-answers.tsv', sep='\t')

In [48]:
#from functools import cache
import re


topic_answer_dict = {}

for language in languages:
    count_errors = 0
    count_found = 0
    for topic in df.columns:
        topic_answer_dict[topic] = []
        for answer in df[topic].values:
            if str(answer) == 'nan':
                continue
            dbpedia_uri = "http://dbpedia.org/resource/" + str(answer)
            #print(label_query.format(uri=dbpedia_uri, lang=language))
            
            sameAs_query_parameterized = sameAs_query.format(uri=dbpedia_uri, lang=language)
            if sameAs_query_parameterized in request_cache:
                result = request_cache[sameAs_query_parameterized]
            else:
                result = execute(sameAs_query_parameterized)
                request_cache[sameAs_query_parameterized] = result
            
            try:
                try:
                    sameAs = result['results']['bindings'][0]['sameAs']['value']
                except Exception as e:
                    if str(answer) != "nan":    
                        print("sameAs query failed:", sameAs_query_parameterized)
                    raise e
                label = get_label_from_wikidata(sameAs, language)
                topic_answer_dict[topic].append({
                    "topic": topic,
                    "topic_translated": get_translation_of_topic(language, topic),
                    "uri": dbpedia_uri,
                    "label": label
                })
                count_found += 1
            except:
                if str(answer) != "nan":
                    count_errors += 1
                    print("errors:", count_errors, "found:", count_found, "No", language, "label:", topic, "answer:", f"'{answer}'")
    save_json(topic_answer_dict, f'../data/mturk/test-topics-answers-{language}.json')
    print(count_errors, "errors for", language)

no en label for http://www.wikidata.org/entity/Q778412 
 
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
SELECT ?label
WHERE {
    <http://www.wikidata.org/entity/Q778412> rdfs:label ?label .
    FILTER (lang(?label) = 'en')    

    <http://www.wikidata.org/entity/Q778412> rdfs:label ?label_en .
    FILTER (lang(?label_en) = 'en')    
    <http://www.wikidata.org/entity/Q778412> rdfs:label ?label_de .
    FILTER (lang(?label_de) = 'de')    
    <http://www.wikidata.org/entity/Q778412> rdfs:label ?label_ru .
    FILTER (lang(?label_ru) = 'ru')    
}

errors: 1 found: 4 No en label: Museum answer: 'Pro_Football_Hall_of_Fame'
no en label for http://www.wikidata.org/entity/Q35732011 
 
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
SELECT ?label
WHERE {
    <http://www.wikidata.org/entity/Q35732011> rdfs:label ?label .
    FILTER (lang(?label) = 'en')    

    <http://www.wikidata.org/entity/Q35732011> rdfs:label ?label_en .
    FILTER (lang(?label_en) = 'en')    
    <http://

# Convert to the MTurk format

In [49]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [50]:
from pprint import pprint
import random
import csv

def add_random_honeypot_options(list_of_real_questions, expected_number_of_real_questions_per_task, language, expected_number_of_honeypot_options, random):
    
    dummy_task = {
        "topic": "honeypot",
        "topic_translated": "honeypot",
        "uri": "honeypot",
        "label": "honeypot"
    }
    
    extended_list_of_questions = []
    
    chunked_list_of_real_questions = list(chunks(list_of_real_questions, expected_number_of_real_questions_per_task))
    
    
    # read JSON file with all possible honeypot options
    with open(f'../data/honeypot-data.json') as f:
        honeypot_data = json.load(f)

        # add {expected_number_of_honeypot_options} to each chunk and append the elements to the extended_list_of_questions
        for chunk in chunked_list_of_real_questions:
            
            for i in range(expected_number_of_honeypot_options):
                index = random.randint(0, len(chunk)) # random index in the chunk for the honeypot option

                honeypot_topic = list(honeypot_data.keys())[random.randint(0, len(list(honeypot_data.keys()))-1)] # find one random topic
                honeypot_topic_translated = get_translation_of_topic(language, honeypot_topic)
                label = honeypot_data[honeypot_topic][language][random.randint(0, len(honeypot_data[honeypot_topic])-1)] # find one random option
                uri = "http://dbpedia.org/resource/H/" + label
                
                assert honeypot_topic != None and honeypot_topic_translated != None and label != None and uri != None
                
                honeypot_task = {
                    "topic": honeypot_topic,
                    "topic_translated": honeypot_topic_translated,
                    "uri": uri,
                    "label": label
                }

                chunk = chunk[:index] + [honeypot_task] + chunk[index:] # insert honeypot option at random index
            
            extended_list_of_questions.extend(chunk)
    
    return extended_list_of_questions


# test
add_random_honeypot_options(["","","",""], 2, "en", 2, random)


['',
 {'topic': 'Soccer club',
  'topic_translated': 'Soccer clubs',
  'uri': 'http://dbpedia.org/resource/H/Houston Rockets',
  'label': 'Houston Rockets'},
 {'topic': 'Magazine',
  'topic_translated': 'Magazines',
  'uri': 'http://dbpedia.org/resource/H/The Scarlet Letter',
  'label': 'The Scarlet Letter'},
 '',
 '',
 {'topic': 'Lake with the area more than 100 square km',
  'topic_translated': 'Lakes with the area more than 100 square km',
  'uri': 'http://dbpedia.org/resource/H/Chesapeake',
  'label': 'Chesapeake'},
 {'topic': 'Soccer club',
  'topic_translated': 'Soccer clubs',
  'uri': 'http://dbpedia.org/resource/H/Houston Rockets',
  'label': 'Houston Rockets'},
 '']

In [51]:
expected_number_of_real_questions_per_task = 8 # this is the number of the NON-fake questions per task

insert_honeypot_options = True
expected_number_of_honeypot_options = 2 # this is the number of the FAKE questions per task

complete_number_of_questions_per_task = expected_number_of_real_questions_per_task + expected_number_of_honeypot_options

if not insert_honeypot_options:
    expected_number_of_honeypot_options = 0 # reset to 0 if honeypot options are not inserted
    
print("crowdsourcing tasks will have", complete_number_of_questions_per_task, "questions", "with", expected_number_of_real_questions_per_task, "real questions and", expected_number_of_honeypot_options, "fake questions")

seed = 42
number_of_attributes_per_task = 4

mturk_header = []
for i in range(complete_number_of_questions_per_task):
    mturk_header.append(f"question_topic_{i}")
    mturk_header.append(f"question_topic_translated_{i}")
    mturk_header.append(f"question_answer_uri_{i}")
    mturk_header.append(f"question_answer_text_{i}")


for language in languages:
    mturk_table = []
    create_shuffled_list_of_tasks = []
    flattened_list = []
    topic_answer_dict = load_json(f'../data/mturk/test-topics-answers-{language}.json')

    for topic, answers in topic_answer_dict.items():
        for answer in answers:
            create_shuffled_list_of_tasks.append(answer)

    # shuffle list, s.t., crowd workers will not see the same topic in one task    
    random.Random(seed).shuffle(create_shuffled_list_of_tasks)

    # if the list is not filling all tasks equally, MTurk would break. Hence, extend the last row.
    number_of_empty_leftovers = expected_number_of_real_questions_per_task - (len(create_shuffled_list_of_tasks) % expected_number_of_real_questions_per_task)
    print(len(create_shuffled_list_of_tasks), "elements.", "found", number_of_empty_leftovers, "empty slots at the end of the table.")
    for i in range(number_of_empty_leftovers):
        # add the first elements to the end of the list
        create_shuffled_list_of_tasks.append(create_shuffled_list_of_tasks[i])

    ##############################################################################################
    # after this point, the list is filled with {mturk_question_numbers} non-fake elements per row

    # add honeypot options to the list, randomly (not an in-place operation)
    if insert_honeypot_options:
        create_shuffled_list_of_tasks = add_random_honeypot_options(create_shuffled_list_of_tasks, expected_number_of_real_questions_per_task, language, expected_number_of_honeypot_options, random)
    
    ##############################################################################################
    # after this point, the list is filled with {expected_number_of_honeypot_options} additional FAKE elements per row

    # create a complete list of tasks
    flattened_list = mturk_header
    for task in create_shuffled_list_of_tasks:
        flattened_list.append(task["topic"])
        flattened_list.append(task["topic_translated"])
        flattened_list.append(task["uri"])
        flattened_list.append(task["label"])
        

    mturk_table = list(chunks(flattened_list, complete_number_of_questions_per_task * number_of_attributes_per_task))
    
    print(len(mturk_table), " tasks in", language, "Mturk table")
    
    #pprint(mturk_table)
        
    csv_filename = f"../data/mturk/mturk-{language}.csv"
    with open(csv_filename,"w+") as my_csv:
        csvWriter = csv.writer(my_csv,delimiter=',', quotechar="\"", quoting=csv.QUOTE_ALL)
        csvWriter.writerows(mturk_table)
    print(language, "exported to", csv_filename)

crowdsourcing tasks will have 10 questions with 8 real questions and 2 fake questions
296 elements. found 8 empty slots at the end of the table.
39  tasks in en Mturk table
en exported to ../data/mturk/mturk-en.csv


In [52]:
for language in languages:
    answer_key = "INPUT:answer_1"
    question_key = "INPUT:question_1"
    
    tasks = []
    topic_answer_dict = load_json(f'../data/mturk/test-topics-answers-{language}.json')

    print("")
    for topic, answers in topic_answer_dict.items():
        print(len(answers), "\t", topic)
        if len(answers) > 0:
            for i in range(len(answers)):
                task_dict = {}
                task_dict[question_key] = topic
                answer = answers.pop()
                task_dict[answer_key] = answer["label"]
                tasks.append(task_dict)
        else:
            continue
        
    save_json(tasks, f"../data/mturk/tasks-test-{language}.json")


17 	 Museum
13 	 Natural disaster
17 	 Beverage
22 	 Active political party
19 	 ColdWar politician
19 	 WWII politician
22 	 Active political parties(last 20 years)
12 	 Film festival
21 	 Soccer club
11 	 Newspaper
25 	 Male actor born after 1950
18 	 Magazine
17 	 Company
15 	 Lake with the area more than 100 square km
15 	 Mountain higher than 4000m
18 	 Nominee of the Academy Award for Best Actor
15 	 News presenter
